<a href="https://colab.research.google.com/github/Leon-web-net/Learning_ML/blob/Notebooks/Regression/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset_path = "./drive/MyDrive/datasets/credit-g.csv"
df = pd.read_csv(dataset_path)
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951.0,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096.0,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870.0,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


# Preprocessing/ Feature encoding

In [ ]:
df["class"].value_counts(),df["class"].value_counts(normalize=True)

(class
 good    700
 bad     300
 Name: count, dtype: int64,
 class
 good    0.7
 bad     0.3
 Name: proportion, dtype: float64)

In [ ]:
y = df["class"]
X = df.drop("class", axis=1)
X.shape, y.shape

((1000, 20), (1000,))

In [ ]:
y_encoded = y.map({"bad":0, "good":1})
y_encoded.head()

,class
0,1
1,0
2,1
3,1
4,0


In [ ]:
encoding_dict = {col: X[col].unique()
                 for col in X.columns
                 if pd.api.types.is_object_dtype(X[col]) or isinstance(X[col].dtype, pd.CategoricalDtype)}
encoding_dict

{'checking_status': array(['<0', '0<=X<200', 'no checking', '>=200'], dtype=object),
 'credit_history': array(['critical/other existing credit', 'existing paid',
        'delayed previously', 'no credits/all paid', 'all paid'],
       dtype=object),
 'purpose': array(['radio/tv', 'education', 'furniture/equipment', 'new car',
        'used car', 'business', 'domestic appliance', 'repairs', 'other',
        'retraining'], dtype=object),
 'savings_status': array(['no known savings', '<100', '500<=X<1000', '>=1000', '100<=X<500'],
       dtype=object),
 'employment': array(['>=7', '1<=X<4', '4<=X<7', 'unemployed', '<1'], dtype=object),
 'personal_status': array(['male single', 'female div/dep/mar', 'male div/sep',
        'male mar/wid'], dtype=object),
 'other_parties': array(['none', 'guarantor', 'co applicant'], dtype=object),
 'property_magnitude': array(['real estate', 'life insurance', 'no known property', 'car'],
       dtype=object),
 'other_payment_plans': array(['none', 'bank', 

In [ ]:
encoding_list = list(encoding_dict.keys())
encoding_list

['checking_status',
 'credit_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job',
 'own_telephone',
 'foreign_worker']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def OE_func(df, col,cats):
    oe = OrdinalEncoder(categories=[cats])
    df[col] = oe.fit_transform(df[[col]]).astype(int)
    return oe

oe_dict = {}

In [ ]:
ordinal_list = ["checking_status","credit_history","savings_status"]
ordinal_category = {"checking_status": ['<0', 'no checking','0<=X<200', '>=200'],
                    "credit_history":['critical/other existing credit','delayed previously', 'existing paid','no credits/all paid', 'all paid'],
                    "savings_status":['no known savings', '<100','100<=X<500', '500<=X<1000', '>=1000' ]}

for col in ordinal_list:
    if isinstance(X[col].dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(X[col]):
       oe_dict[col] = OE_func(X, col,ordinal_category[col])
    else:
      print(f"{col} is of {X[col].dtype} and has not been labelEncoded")

X.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,0,6,0,radio/tv,1169.0,0,>=7,4,male single,none,4,real estate,67,none,own,2,skilled,1,yes,yes
1,2,48,2,radio/tv,5951.0,1,1<=X<4,2,female div/dep/mar,none,2,real estate,22,none,own,1,skilled,1,none,yes
2,1,12,0,education,2096.0,1,4<=X<7,2,male single,none,3,real estate,49,none,own,1,unskilled resident,2,none,yes
3,0,42,2,furniture/equipment,7882.0,1,4<=X<7,2,male single,guarantor,4,life insurance,45,none,for free,1,skilled,2,none,yes
4,0,24,1,new car,4870.0,1,1<=X<4,3,male single,none,4,no known property,53,none,for free,2,skilled,2,none,yes


In [ ]:
for col, oe in oe_dict.items():
    mapping = {label: idx for idx, label in enumerate(oe.categories_[0])}
    print(f"{col}: {mapping}")

checking_status: {'<0': 0, 'no checking': 1, '0<=X<200': 2, '>=200': 3}
credit_history: {'critical/other existing credit': 0, 'delayed previously': 1, 'existing paid': 2, 'no credits/all paid': 3, 'all paid': 4}
savings_status: {'no known savings': 0, '<100': 1, '100<=X<500': 2, '500<=X<1000': 3, '>=1000': 4}


In [ ]:
OHE_list = [col for col in encoding_list if col not in ordinal_list]
OHE_list

['purpose',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job',
 'own_telephone',
 'foreign_worker']

In [ ]:
X = pd.get_dummies(X, columns=OHE_list, drop_first=True)
X.head()

,checking_status,duration,credit_history,credit_amount,savings_status,installment_commitment,residence_since,age,existing_credits,num_dependents,...,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_skilled,job_unemp/unskilled non res,job_unskilled resident,own_telephone_yes,foreign_worker_yes
0,0,6,0,1169.0,0,4,4,67,2,1,...,True,True,False,True,False,True,False,False,True,True
1,2,48,2,5951.0,1,2,2,22,1,1,...,True,True,False,True,False,True,False,False,False,True
2,1,12,0,2096.0,1,2,3,49,1,2,...,True,True,False,True,False,False,False,True,False,True
3,0,42,2,7882.0,1,2,4,45,1,2,...,False,True,False,False,False,True,False,False,False,True
4,0,24,1,4870.0,1,3,4,53,2,2,...,False,True,False,False,False,True,False,False,False,True


In [ ]:
X.shape

(1000, 40)

# Scaling

In [ ]:
X.head()

,checking_status,duration,credit_history,credit_amount,savings_status,installment_commitment,residence_since,age,existing_credits,num_dependents,...,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_skilled,job_unemp/unskilled non res,job_unskilled resident,own_telephone_yes,foreign_worker_yes
0,0,6,0,1169.0,0,4,4,67,2,1,...,True,True,False,True,False,True,False,False,True,True
1,2,48,2,5951.0,1,2,2,22,1,1,...,True,True,False,True,False,True,False,False,False,True
2,1,12,0,2096.0,1,2,3,49,1,2,...,True,True,False,True,False,False,False,True,False,True
3,0,42,2,7882.0,1,2,4,45,1,2,...,False,True,False,False,False,True,False,False,False,True
4,0,24,1,4870.0,1,3,4,53,2,2,...,False,True,False,False,False,True,False,False,False,True


In [ ]:
# print(f"dtypes\n{X.dtypes}")
summary = pd.DataFrame({
    'dtype': X.dtypes,
    'max': X.max()
})
summary = summary[summary['dtype'] != 'bool']
summary_list = summary.index.tolist()

X[summary_list].head()

,checking_status,duration,credit_history,credit_amount,savings_status,installment_commitment,residence_since,age,existing_credits,num_dependents
0,0,6,0,1169.0,0,4,4,67,2,1
1,2,48,2,5951.0,1,2,2,22,1,1
2,1,12,0,2096.0,1,2,3,49,1,2
3,0,42,2,7882.0,1,2,4,45,1,2
4,0,24,1,4870.0,1,3,4,53,2,2


In [ ]:
scale_list = ["duration", "credit_amount", "age"]

# Logistic Regression



## IRLS for Logistic Regression

**1. Logistic model:**
$$
p = \sigma(X \beta), \quad \sigma(t) = \frac{1}{1 + e^{-t}}
$$

**2. Current iteration $t$:**
- Parameters: $\beta^{(t)}$
- Predicted probabilities: $p^{(t)} = \sigma(X \beta^{(t)})$
- Weights: $W = \mathrm{diag}\big(p^{(t)} (1 - p^{(t)})\big)$
- Pseudo-response:
$$
z = X \beta^{(t)} + W^{-1} (y - p^{(t)})
$$

**3. Weighted least squares step:**
$$
\beta^{(t+1)} = \arg\min_\beta \| W^{1/2} (z - X \beta) \|^2
$$

**4. Closed-form solution (normal equations):**
$$
\beta^{(t+1)} = (X^\top W X)^{-1} X^\top W z
$$

**5. Iterate until convergence:**
- $||\beta^{(t+1)} - \beta^{(t)}|| < \epsilon$  
- or $\ell(\beta^{(t+1)}) - \ell(\beta^{(t)}) < \epsilon$

**Intuition:**
- $z$ = current prediction + scaled residuals (pseudo-target)  
- $W$ = uncertainty-based weights (points near $p=0.5$ have more influence)  
- Solving weighted L2 moves $\beta$ closer to the maximum likelihood estimate (MLE)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# scaler = StandardScaler()

# # Fit only on training set, transform both train and test
# X_train[scale_list] = scaler.fit_transform(X_train[scale_list])
# X_test[scale_list] = scaler.transform(X_test[scale_list])


# Convert to NumPy arrays
X_train = X_train.to_numpy(dtype=float)
X_test  = X_test.to_numpy(dtype=float)
y_train = y_train.to_numpy(dtype=float)
y_test  = y_test.to_numpy(dtype=float)

In [ ]:
y_train[0]

np.float64(1.0)

In [ ]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [ ]:
def logistic_regression_newton(X, y, max_iter=20, tol=1e-6):
    """
    Logistic regression via Newton-Raphson (IRLS).
    Args:
        X: (n_samples, n_features) numpy array
        y: (n_samples,) numpy array, binary {0,1}
        max_iter: max number of iterations
        tol: convergence tolerance
    Returns:
        beta: estimated coefficients
    """

    # Add intercept column
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    n, d = X.shape

    # Initialise beta
    beta = np.zeros(d)

    for t in range(max_iter):
        # Linear predictor
        eta = X @ beta
        # Predicted probabilities
        p = sigmoid(eta)

        # Gradient (score function)
        grad = X.T @ (y - p)

        # Hessian
        W = np.diag(p * (1 - p))
        H = -(X.T @ W @ X)

        # Newton-Raphson update
        step = np.linalg.solve(-H, grad)   # same as inv(H) @ grad
        beta_new = beta + step

        # Check convergence
        if np.max(np.abs(beta_new - beta)) < tol:
            print(f"Converged in {t+1} iterations")
            beta = beta_new
            break

        beta = beta_new

    return beta

In [ ]:
beta = logistic_regression_newton(X_train, y_train)

# Predict on test
X_test_ = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
p_test = sigmoid(X_test_ @ beta)
y_pred = (p_test >= 0.5).astype(int)

from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
accuracy = np.mean(y_pred == y_test)
print(f"accuracy: {accuracy}")
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Classification report (precision, recall, f1 for each class)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Good (0)", "Bad (1)"]))

Converged in 6 iterations
accuracy: 0.675
Confusion Matrix:
 [[ 24  36]
 [ 29 111]]

Classification Report:
               precision    recall  f1-score   support

    Good (0)       0.45      0.40      0.42        60
     Bad (1)       0.76      0.79      0.77       140

    accuracy                           0.68       200
   macro avg       0.60      0.60      0.60       200
weighted avg       0.66      0.68      0.67       200



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

skl_LR= LogisticRegression()
Logi_fit = skl_LR.fit(X_train, y_train)
y_pred = Logi_fit.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Good (0)", "Bad (1)"]))

Test accuracy: 0.735
Confusion Matrix:
 [[ 26  34]
 [ 19 121]]

Classification Report:
               precision    recall  f1-score   support

    Good (0)       0.58      0.43      0.50        60
     Bad (1)       0.78      0.86      0.82       140

    accuracy                           0.73       200
   macro avg       0.68      0.65      0.66       200
weighted avg       0.72      0.73      0.72       200



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Scaling issue:

 In this case Scaling hurt performance:

When you scale only some features, you're creating an **artificial imbalance** in feature importance:

### Without scaling:

- credit_amount might range from 1,000 to 20,000 (high magnitude)
- age ranges from 18 to 80 (medium magnitude)
- duration ranges from 1 to 60 (medium magnitude)
- Encoded categoricals are 0, 1, 2, 3... (low magnitude)

### With scaling:

- credit_amount, age, duration all become ~[-2, 2] (normalized)
- Encoded categoricals remain 0, 1, 2, 3... (now relatively large!)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Good (0)", "Bad (1)"]))

Test accuracy: 0.725
Confusion Matrix:
 [[ 19  41]
 [ 14 126]]

Classification Report:
               precision    recall  f1-score   support

    Good (0)       0.58      0.32      0.41        60
     Bad (1)       0.75      0.90      0.82       140

    accuracy                           0.72       200
   macro avg       0.67      0.61      0.61       200
weighted avg       0.70      0.72      0.70       200

